<a href="https://colab.research.google.com/github/tinycaterpillar/Ai-project/blob/HwangJinIk/main_gru.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### 베이스라인은 참고용 코드 입니다. 여러분 만의 고도화된 모델을 만들어 주세요

## Hyperparameter Setting

In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

#하이퍼 파라미터
hyperparam = {
    'epoch':50,
    'learning_rate':1e-3,
    'batch_size':128,
}
constant = {
    'day_to_predict': 2, # 예측해야 하는 day의 수
    'total_day' : 200
}

## Colab drive mount

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 데이터 불러오기

In [ ]:
Path = "/content/drive/MyDrive/Colab Notebooks/2022 Inha challenge/data/"
train_data = np.load(Path+"train_data.npy")
train_labels = np.load(Path+"train_labels.npy")
validation_data = np.load(Path+"validation_data.npy")
validation_labels = np.load(Path+"validation_labels.npy")

## 현재 모델 저장용 directory 생성

In [ ]:
import datetime
import os

serial = f"{datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}"
directory = f"/content/drive/MyDrive/Colab Notebooks/2022 Inha challenge/model/{serial}"
os.makedirs(directory)

## Custom Dataset

In [ ]:
class CustomDataset:
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels
    
    def __len__(self):
        return len(self.labels)

    def __getitem__(self, ind):
        return torch.Tensor(self.data[ind]), torch.Tensor(self.labels[ind])

## DataLoader setting

In [ ]:
train_dataset = CustomDataset(train_data, train_labels)
validation_dataset = CustomDataset(validation_data, validation_labels)
train_loader = DataLoader(train_dataset, batch_size = hyperparam['batch_size'], shuffle=True)

train_loader = DataLoader(
    dataset = train_dataset,
    batch_size = hyperparam["batch_size"],
    shuffle = True
)
test_loader = DataLoader(
    dataset = validation_dataset,
    batch_size = hyperparam["batch_size"],
    shuffle = False
)

## 모델 선언

In [ ]:
class BaseModel(nn.Module):
    def __init__(self):
        super(BaseModel, self).__init__()
        self.gru = nn.GRU(input_size=8, hidden_size=256, num_layers = 2, batch_first=True, bidirectional=False)
        self.classifier = nn.Sequential(
            nn.Linear(256,516),
            nn.ReLU(),
            nn.Linear(516,288),
            nn.ReLU()
        )
        
    def forward(self, x):
        hidden, _ = self.gru(x)
        output = self.classifier(hidden[:,-1,:])
        return output

# 평가 함수

## Customize loss function
https://neptune.ai/blog/pytorch-loss-functions

In [ ]:
def lossfunction(input, target, day = 2):
    return (torch.sqrt(torch.sum((input - target)**2)/(day*144)) + torch.sum(torch.abs(input - target))/(day*144))/2

## 모델 테스트

In [ ]:
def test():
    model.eval() # nn.Module.eval method, train 과 test 를 구분, train 할 때는 model.train 을 호출한다
    score = 0
    maxval = 0
    minval = 1e9
    for data, target in test_loader:
        data, target = data.to(device), target.to(device) # gpu 에 tensor 들을 할당함(cpu가 아닌 gpu 로 계산하기 위함), 여기서 data, target 은 tensor 이다
        output = model(data)

        cur = lossfunction(output, target)
        score += cur
        maxval = max(maxval, cur)
        minval = min(minval, cur)

    score /= len(test_loader)
    print("="*44)
    print(f"Test set: socore : {score:.4f}")
    print(f"        : max    : {maxval:.4f}")
    print(f"        : min    : {minval:.4f}")

## 모델 학습

In [ ]:
def train():
    model.to(device)
    for epoch in range(1, hyperparam['epoch']+1):
        model.train()
        train_loss = []
        for X, Y in tqdm(iter(train_loader)):
            X = X.to(device)
            Y = Y.to(device)
            
            optimizer.zero_grad()
            
            output = model(X)
            loss = lossfunction(output, Y)
            loss.backward()
            optimizer.step()
            
            train_loss.append(loss.item())
        
        print(f'Epoch : [{epoch}] Train Loss : [{np.mean(train_loss):.5f}]')
        test()
        torch.save(model.state_dict(), directory+f"{epoch}.pth", _use_new_zipfile_serialization=False)
        print('Model Saved.')

### 모델 학습

In [ ]:
model = BaseModel()
model.eval()
optimizer = torch.optim.Adam(params = model.parameters(), lr = hyperparam["learning_rate"])
train()

100%|██████████| 163/163 [00:27<00:00,  6.01it/s]


Epoch : [1] Train Loss : [22367.94096]
Test set: socore : 21007.0703
        : max    : 23312.7500
        : min    : 12909.7734
Model Saved.


100%|██████████| 163/163 [00:27<00:00,  5.99it/s]


Epoch : [2] Train Loss : [20361.37977]
Test set: socore : 19936.3809
        : max    : 22127.4863
        : min    : 12344.8105
Model Saved.


100%|██████████| 163/163 [00:27<00:00,  6.02it/s]


...

100%|██████████| 163/163 [00:27<00:00,  6.02it/s]


Epoch : [16] Train Loss : [13233.79737]
Test set: socore : 13682.2920
        : max    : 15294.6553
        : min    : 8638.5107
Model Saved.


100%|██████████| 163/163 [00:27<00:00,  6.00it/s]


Epoch : [17] Train Loss : [13047.29232]
Test set: socore : 13179.5381
        : max    : 14873.2236
        : min    : 8441.7148
Model Saved.


  9%|▊         | 14/163 [00:02<00:25,  5.96it/s]